In [1]:
from map_manager.map_generator import map
from map_manager.noise import toddler
from map_manager.world_encounters import *

In [3]:
nw = map()

nw.populate_map(n_locations=1024, name="terrain",relaxed=True,k=100)
nw.populate_map(n_locations=1024, name="civilisation",relaxed=True,k=100)
nw.populate_map(n_locations=1024, name="threat",relaxed=True, k=100)
nw.populate_map(n_locations=1024, name="faction",relaxed=True, k=100)


nw.attribute_view(seed_1=20,seed_2=30,map_name="terrain",view_name="terrain")
nw.attribute_view(seed_1=40,seed_2=50,map_name="civilisation",view_name="civilisation")
nw.attribute_view(seed_1=34,seed_2=34,map_name="threat",view_name="threat",double=False)
nw.attribute_view(seed_1=67,seed_2=89,map_name="faction",view_name="faction",double=True)

/Users/cetiners/Desktop/Thesis/human_error/map_manager/map_attributer.py:84: RuntimeWarning: invalid value encountered in true_divide
  average = sum_/count


array([[4., 4., 4., ..., 1., 1., 1.],
       [4., 4., 4., ..., 1., 1., 1.],
       [4., 4., 4., ..., 1., 1., 1.],
       ...,
       [4., 4., 4., ..., 4., 4., 4.],
       [4., 4., 4., ..., 4., 4., 4.],
       [4., 4., 4., ..., 4., 4., 4.]])

In [10]:
from map_manager.world_encounters import *
from genetic_algorithm.crossover import * 

p1 = pack(nw,type="camel",size=20,encounter_type="friendly_animals")

p2 =  pack(nw,type="camel",size=20,encounter_type="friendly_animals")

o1,o2 = partially_mapped_xo(p1,p2)

it = 0 

while o1.pack_fitness < 0:
    print(f"Iteration number:{it}, Pack fitness:{o1.pack_fitness}")
    o1,o2 = partially_mapped_xo(o1,o2)
    it += 1
    if it // 1000:
        o2 = pack(nw,type="camel",size=20,encounter_type="friendly_animals")


Iteration number:0, Pack fitness:-32000
Iteration number:1, Pack fitness:-33000
Iteration number:2, Pack fitness:-33000
Iteration number:3, Pack fitness:-34000
Iteration number:4, Pack fitness:-34000
Iteration number:5, Pack fitness:-34000
Iteration number:6, Pack fitness:-34000
Iteration number:7, Pack fitness:-34000
Iteration number:8, Pack fitness:-34000
Iteration number:9, Pack fitness:-33000
Iteration number:10, Pack fitness:-33000
Iteration number:11, Pack fitness:-33000
Iteration number:12, Pack fitness:-33000
Iteration number:13, Pack fitness:-33000
Iteration number:14, Pack fitness:-33000
Iteration number:15, Pack fitness:-33000
Iteration number:16, Pack fitness:-33000
Iteration number:17, Pack fitness:-33000
Iteration number:18, Pack fitness:-33000
Iteration number:19, Pack fitness:-33000
Iteration number:20, Pack fitness:-33000
Iteration number:21, Pack fitness:-33000
Iteration number:22, Pack fitness:-33000
Iteration number:23, Pack fitness:-33000
Iteration number:24, Pack 

IndexError: index 1024 is out of bounds for axis 0 with size 1024

In [ ]:
from genetic_algorithm.mutation import * 

pop = pack_population(nw,pop_size=100,type="camel",pack_size=25,encounter_type="friendly_animals")


In [ ]:

civ = nw.views["civilisation"].copy()
ter = nw.views["terrain"].copy()
thr = nw.views["threat"].copy()
fac = nw.views["faction"].copy()

for i in range(1024):
    for j in range(1024):
        civ[i,j] += 1
        ter[i,j] += 1
        thr[i,j] += 1
        fac[i,j] += 1

import matplotlib.pyplot as plt
import numpy as np 

fig, ax = plt.subplots(1,4)
fig.set_dpi(150)
fig.set_size_inches(18, 6)

import random
seed=random.randint(1,100)
res=2
octaves = 15
persistence = 0.60
lacunarity = 2
mask = toddler(size=1024, seed=seed,res=res, octaves = octaves, persistence = persistence, lacunarity = lacunarity,mask=True)

ax[0].imshow(civ*(np.invert(mask)),cmap="Greens")
ax[0].set_title("Civilisation")

ax[1].imshow(ter*(np.invert(mask)),cmap="Blues")
ax[1].set_title("Terrain")

ax[2].imshow(thr*(np.invert(mask)),cmap="Reds")
ax[2].set_title("Threat")

ax[3].imshow(fac*(np.invert(mask)),cmap="Greys")
ax[3].set_title("Factions")

In [ ]:
[i for i in range(3)]

In [ ]:
random.random()